In [527]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column


# Horner Division Table


## Algorithm

- Objective:
Given a polynomial $p(x) = a_0 + a_1x + a_2x^2 + \dots + a_nx^n$  
and a constant $c$, we compute:
    - The value $p(c)$
    - The coefficients of the quotient $q(x) = \frac{p(x)}{x - c}$

- Input
    - A list of coefficients $[a_0, a_1, \dots, a_n]$
    - A real number $c$

- Output
    - Value of $p(c)$
    - Coefficients $[b_1, b_2, \dots, b_n]$ of $q(x)$
    - Table showing $a_i$, $b_i \cdot c$, and $b_i$

- Steps
1. Initialize $b_n = a_n$
2. For $i = n-1$ down to $0$:
   - Compute $b_i = a_i + b_{i+1} \cdot c$
3. The value of $p(c)$ is $b_0$
4. The coefficients of $q(x)$ are $[b_1, b_2, \dots, b_n]$

In [528]:
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

## Using in finding all derivatives at a given point

- Objective: 
Given a polynomial  $p(x) = a_0 + a_1x + a_2x^2 + \dots + a_nx^n$,  
we want to find all derivatives $p^{(i)}(c)$ at a specific point $x = c$  
using Horner’s method recursively.

- Input
    - List of coefficients $[a_0, a_1, \dots, a_n]$
    - A real number $c$

- Output: A table of values:
  - $i$: derivative order
  - $b_0$: result of the $i$-th Horner division
  - $p^{(i)}(c)$: the actual derivative value at $x=c$

- Steps
1. Initialize the current coefficient list as $[a_0, a_1, \dots, a_n]$  
   and set $i = 0$.
2. Call **Horner’s division** to compute:
   - $b_0 = p^{(i)}(c) / i!$
   - The new coefficient list $[b_1, b_2, \dots, b_n]$ for $Q(x)$.
3. Compute the true derivative value:
   $p^{(i)}(c) = b_0 \cdot i!$
4. Store $(i, b_0, p^{(i)}(c))$.
5. Replace the current coefficients with $[b_1, b_2, \dots, b_n]$.
6. Increase $i$ by 1 and repeat Steps 2–5 until no coefficients remain.
7. Output all computed derivatives in a table.

In [529]:
def all_derivatives(a, c):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

## Result


In [530]:
# Example polynomial: p(x) = 3x^3 - 2x - 1
a = [-46, -5, 4, -3, 2, 1]   # coefficients of p(x)
c = 2          # value at which to evaluate


In [531]:
df, p_c, q_coeff, b = synthetic_division(a, c)
df.style.hide(axis="index")

i,a_i,b_i*c,b_i = a_i + b_(i+1)*c
5,1,2,1
4,2,8,4
3,-3,10,5
2,4,28,14
1,-5,46,23
0,-46,,0


In [532]:
df2 = all_derivatives(a, c)
df2.style

,a_i,i=0,i=1,i=2,i=3,i=4,i=5
c,2.000000,nan,nan,nan,nan,nan,nan
0,-46.000000,0.000000,119.000000,114.000000,53.000000,12.000000,1.000000
1,-5.000000,23.000000,48.000000,33.000000,10.000000,1.000000,nan
2,4.000000,14.000000,17.000000,8.000000,1.000000,nan,nan
3,-3.000000,5.000000,6.000000,1.000000,nan,nan,nan
4,2.000000,4.000000,1.000000,nan,nan,nan,nan
5,1.000000,1.000000,nan,nan,nan,nan,nan
b_0,nan,0.000000,119.000000,114.000000,53.000000,12.000000,1.000000
p^(i)(c),nan,0.000000,119.000000,228.000000,318.000000,288.000000,120.000000


# Horner Multiplication Table

## Algorithm


- Objective:

Given the sequence of $b_i$ coefficients obtained from the forward Horner (synthetic division) process with evaluation point $c$, reconstruct the original polynomial coefficients $a_i$.

- Concept:

From Horner’s method, we know:
$
b_{i} = a_{i} + b_{i+1} \cdot c
$
where $b_n = a_n$ (since the last coefficient remains the same).

To reverse this relation and find $a_i$, we can rearrange:
$
a_{i} = b_{i} - b_{i+1} \cdot c
$

We can iteratively compute all $a_i$ starting from the highest degree down to the constant term.

- Steps

1. **Input:**  
   - List of $b$ coefficients: $[b_0, b_1, \dots, b_n]$  
   - Evaluation point: $c$

2. **Initialization:**  
   - Let $a_n = b_n$

3. **For each** $i$ from $n-1$ down to $0$:  
   - Compute  
     $$
     a_i = b_i - b_{i+1} \cdot c
     $$

4. **Output:**  
   - List of coefficients $[a_0, a_1, \dots, a_n]$ representing the original polynomial.

In [533]:
def reverse_horner(b, c):
    """
    Reverse Horner method:
    Given b_i coefficients of q(x) and point c,
    reconstruct a_i coefficients of p(x) = (x - c) * q(x)
    """

    n = len(b)
    b = [0] + b;
    a = [0] * (n + 1)

    # highest term: a_n = b_n
    a[-1] = b[-1]

    # compute remaining coefficients
    for i in range(n-1, -1, -1):
        a[i] = b[i] - c * b[i+1]

    # Build DataFrame for clear display
    data = {
        "i": list(range(n, -1, -1)),
        "b_i": b[::-1],
        "b_i*c": [c * b[i] for i in range(n, -1, -1)],
        "a_i = b_i - c*b_(i+1)": a[::-1]  # exclude the last term a_n (it's beyond b_i range)
    }

    df = pd.DataFrame(data)

    return df, a

In [534]:
# Example usage
b = [2, 3, 5]  # coefficients of q(x)
c = 4

df, a = reverse_horner(b, c)
df.style.hide(axis="index")

i,b_i,b_i*c,a_i = b_i - c*b_(i+1)
3,5,20,5
2,3,12,-17
1,2,8,-10
0,0,0,-8


## W_function multiplication

- Constructing $ w_{n+1}(x) = \prod_{k=0}^{n} (x - x_k) $

- Input:
A list of $ n+1 $ nodes $ [x_0, x_1, \ldots, x_n] $.

- Output:
The list of coefficients of the polynomial $ w_{n+1}(x) $ in standard form:
$
w_{n+1}(x) = a_0 + a_1 x + a_2 x^2 + \ldots + a_{n+1} x^{n+1}.
$

- Steps
1. Initialize the polynomial $ w(x) = 1 $.
2. For each node $ x_k $ in the list $ [x_0, x_1, \ldots, x_n] $:
   - Multiply the current polynomial $ w(x) $ by $ (x - x_k) $.
   - In coefficient form, if $ w(x) = b_1 + b_2 x + \ldots + b_m x^{m+1} $,  
     then the new polynomial coefficients are computed as:
     $
     a_i = b_{i} - b_{i+1} \, x_k,
     $
     with $ a_{-1} = 0 $ and $ a_{m+1} = 0 $.
3. Update $ w(x) $ with the new coefficients after each multiplication.
4. Repeat until all $ x_k $ have been processed.
5. Return the final list of coefficients of $ w_{n+1}(x) $.

- Note
    - Each step represents one multiplication by $ (x - x_k) $.
    - The coefficients can be stored or displayed in a table to trace how the polynomial expands with each added node.


In [535]:
def w_function(x_points):
    """
    Construct w_{n+1}(x) = Π(x - x_k)
    using Reverse Horner multiplications.
    """
    steps = {}
    
    # Start with first term (x - x_0)
    coeffs = [-x_points[0], 1]
    steps[0] = coeffs[:]
    
    # Multiply recursively for all remaining x_k
    for i in range(1, len(x_points)):
        df, coeffs = reverse_horner(coeffs, x_points[i])
        steps[i] = coeffs[:]
    
    # ---- Create the display table ----
    max_len = max(len(v) for v in steps.values())
    df_data = {}

    for i, coeff_list in steps.items():
        padded = coeff_list + [np.nan] * (max_len - len(coeff_list))
        df_data[f"i={i}"] = padded

    df = pd.DataFrame(df_data)
    df.insert(0, "a_i", [f"coeff_{j}" for j in range(max_len)])
    
    # Add top row for x_k
    df.loc["x_k"] = ["x_k"] + [x for x in x_points] + [np.nan]*(df.shape[1]-len(x_points)-1)
    df = df.loc[["x_k"] + [idx for idx in df.index if idx != "x_k"]]  # Move row to top

    return df, coeffs


In [536]:
x_points = [2, 2.4, 2.7, 3, 3.1, 3.4]

df, final_coeffs = w_function(x_points)
df.style

,a_i,i=0,i=1,i=2,i=3,i=4,i=5
x_k,x_k,2.000000,2.400000,2.700000,3.000000,3.100000,3.400000
0,coeff_0,-2.000000,4.800000,-12.960000,38.880000,-120.528000,409.795200
1,coeff_1,1.000000,-4.400000,16.680000,-63.000000,234.180000,-916.740000
2,coeff_2,nan,1.000000,-7.100000,37.980000,-180.738000,848.689200
3,coeff_3,nan,nan,1.000000,-10.100000,69.290000,-416.324000
4,coeff_4,nan,nan,nan,1.000000,-13.200000,114.170000
5,coeff_5,nan,nan,nan,nan,1.000000,-16.600000
6,coeff_6,nan,nan,nan,nan,nan,1.000000
